<a href="https://colab.research.google.com/github/ryb4666/TugasAkhirFMIPAUSK/blob/main/CEITEC_LIBS_analysis_tutorial_(locked).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Classification of Laser-induced breakdown spectroscopy (LIBS) datasets**

authors:
- Jakub Hruška <jhruska@mail.muni.cz>
- Jakub Vrábel <jakub.vrabel@ceitec.vutbr.cz>

# 1) Introduction and dataset loading

## Environment setup

This notebook leads you through basic data analysis of Laser-Induced Breakdown Spectroscopy (LIBS) data. It sheds light on data processing and machine learning with primary focus on classification.

---

At first you must copy this _read-only_ file into your Google Drive. Then you will be able to run and modify the cells.

_File -> Save a copy in Drive..._  
_File -> Locate in Drive_ (to find the location of your newly created copy)

---

In the next steps you prepare an environment on your Google Drive for convenient work with this tutorial.

Start with creating a Colab runtime environment and mount your Google Drive into this environment. Just run the following cell and go through the auth procedure.

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In _Files_ tab (left menu) you can now see your Google Drive mounted at `drive/My Drive/`. For easier manipulation in the future change your working directory (currently `/content/`) using module [os](https://docs.python.org/3.6/library/os.html). Change `wd_path` variable to a path to a directory in your Google Drive where you want to work.

In [ ]:
import os

wd_path = "COLAB_LIBS/contest"
os.chdir("/content/drive/My Drive/" + wd_path)

_Note:_ There are more options how to upload files into Google Colab. One possible alternative is direct upload from your local drive. `files.upload()` command allows you to upload files directly. It saves them into your current working directory (it can be within your mounted Google Drive also) and returns a dictionary of the files which were uploaded. It is sometimes needed (in some browsers) to run the cell twice.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Both ways (mounting your GDrive or direct upload), you have to repeat the process every time you restart your runtime enviroment. An exhaustive description of how to load and save files in Google Colab is [here](https://colab.research.google.com/notebooks/io.ipynb).

Now you need to get and install our tool for loading EMSLIBS-contest datasets from [GitHub](https://github.com/JVrabel/EMSLIBS_contest).

In [ ]:
!git clone https://github.com/JVrabel/EMSLIBS_contest.git
os.chdir("EMSLIBS_contest")

If you already have done this once, you don't have to clone the GitHub repository again. Just check you are in correct working directory. If there is an upgrade in the tool and you want to update your copy, just use `!git pull` command in the `EMSLIBS_contest` directory.

In [ ]:
os.chdir("EMSLIBS_contest")

In [ ]:
!git pull

Then you simply import the tool. Please, take a look into the in-code [documentation](https://github.com/JVrabel/EMSLIBS_contest/blob/master/load_scripts/h5_load_contest.py) for more details.

In [ ]:
import load_scripts.h5_load_contest as emslibs_load

Finally you need to upload the dataset file into the `datasets/` subfolder. In this tutorial you need only `train.h5` dataset from this [link](https://ndownloader.figstatic.com/files/20065616). You can find more info about the dataset (including original _test_ dataset) in this [paper](https://www.nature.com/articles/s41597-020-0396-8). After storing it in the `datasets/` subfolder at your GDrive, rename it to `contest_TRAIN.h5` please.

You can do this all using this single-line command.

In [ ]:
!wget -O datasets/contest_TRAIN.h5 https://ndownloader.figstatic.com/files/20065616

## Read the files

We use [pandas](https://pandas.pydata.org/) and [numpy](https://numpy.org/) packages for manipulating with our datasets in this tutorial and in the EMSLIBS-contest loading tool.

Check the documentation of Pandas [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) structures, used to store and manipulate the datasets.

In [ ]:
import pandas as pd
import numpy as np

Reading the .h5 file can take a while.

In [ ]:
path_to_ds = 'datasets/'
spectra_count = 100
wavelengths = emslibs_load.load_wavelengths(path_to_ds)
wavelengths = np.round(wavelengths, 2)
x_raw = emslibs_load.load_train_data(path_to_ds, spectra_count)
x_raw = pd.DataFrame(x_raw, columns=wavelengths)
y_raw = emslibs_load.load_train_labels(path_to_ds, spectra_count)
y_raw = pd.Series(y_raw)

For sake of different models comparison it's necessary to split the train dataset into two: _train_ and _validation_. You should use only _train_ dataset for training a model and _validation_ (_val_) dataset for evaluation of the model.

This task (contest from EMSLIBS 2019) has quite specific test dataset (see the paper mentioned before for more info), so ideally your _train-validation_ split should reflect this too. In this simple tutorial, we ignore it and use basic `train_test_split()` from _Scikit-learn_. However, keep in mind that the results are much better in this case, because the datasets are much more similar compared to the original _train-test_ split in the contest.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train_raw, x_val_raw, y_train_raw, y_val_raw = train_test_split(
    x_raw, y_raw, train_size=0.7, random_state=42)
del x_raw, y_raw

# 2) Exploring the dataset

Now we have 4 DataFrames - two datasets (training and validation) divided into inputs `x_` (spectra) and desired outputs `y_` (labels of classes). Plus we have stored values of wavelengths (in nm) presented in the datasets.

At first we check [shape](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shape.html) of our datasets:

In [ ]:
print("Train\t\tx:{}, y:{}".format(x_train_raw.shape, y_train_raw.shape))
print("Validation\tx:{}, y:{}".format(x_val_raw.shape, y_val_raw.shape))
print("Wavelengths:\t{}".format(wavelengths.shape))

You can see that the train dataset contains 7000 samples with 40 002 features (variables, dimensions, attributes, ...) each. Every feature corresponds to certain wavelength. Validation dataset consists of 3000 samples of same dimension as training dataset.

Let's look directly into datasets using [`.head()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html) and [`.tail()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.tail.html) functions:

In [ ]:
print("Train:")
print(x_train_raw.head())
print(x_train_raw.tail())
print("\nValidation:")
print(x_val_raw.head())
print(x_val_raw.tail())

It is also possible to [slice](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#slicing-ranges) pandas DataFrames (and other pandas objects) same way as Python lists. You can find complete documentation of indexing and accessing elements in pandas objects [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html).

In [ ]:
print("Waveleghts:")
print(wavelengths[:8])

Class labels are stored in a [Series](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html) object (DataFrames are collections of Series). You can print it directly:

In [ ]:
print("Train class labels:")
print(y_train_raw)

Or get some summary using [`.value_counts()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html) function. Try to change `normalize` parameter to _True_.

In [ ]:
train_label_counts = y_train_raw.value_counts(normalize=False)
val_label_counts = y_val_raw.value_counts(normalize=False)

print("Train class labels value counts:")
print(train_label_counts)
print("\nValidation class labels value counts:")
print(val_label_counts)

We use [plotly](https://plot.ly/python/) library for plotting visuals and [colorlover](https://github.com/plotly/colorlover) package for choosing color scales (color maps) in this tutorial. You can find various types of colormaps at [ColorBrewer](http://colorbrewer2.org).

In [ ]:
import plotly.graph_objects as go
import colorlover as cl

In [ ]:
counts = pd.DataFrame({'train': train_label_counts, 'validation': val_label_counts})

fig = go.Figure()
for i in range(counts.shape[0]):
    fig.add_trace(
        go.Bar(
            x = counts.columns,
            y = counts.iloc[i],
            name = "Class {}".format(i+1),
            marker = {'color': cl.scales['12']['qual']['Paired'][i]}
        )
    )
fig.update_layout(
    title = "Datasets composition",
    barmode ='stack')
fig.show()

For some basic statistical overview of the DataFrame you can use [`DataFrame.describe()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) function. It gives you statistics variable-wise, but can be time consuming for larger datasets (roughly 3 mins for this validation dataset under default Google Colab runtime environment).

In [ ]:
print(x_val_raw.describe())

Or you can use numpy's [statistics](https://docs.scipy.org/doc/numpy/reference/routines.statistics.html):

In [ ]:
print("--> min\n{}\n".format(np.min(x_train_raw)))
print("--> mean\n{}\n".format(np.mean(x_train_raw)))
print("--> max\n{}\n".format(np.max(x_train_raw)))
print("--> std\n{}\n".format(np.std(x_train_raw)))

And finally let's look how our spectra look like. First we define function for plotting multiple spectra into one plot:

In [ ]:
def plot_spectra(spectra, calibration=None, title=None, labels=None,
                 colormap=cl.scales['12']['qual']['Paired'], axes_titles=True):
    if calibration is None:
        calibration = np.arange(len(spectra[0]))
    if labels is None:
        labels = ["class {}".format(x+1) for x in range(len(spectra))]
    fig = go.Figure()
    for i in range(len(spectra)):
        fig.add_trace(
            go.Scatter(
                x = calibration,
                y = spectra[i],
                name = str(labels[i]),
                line = {'color': colormap[i % len(colormap)]},
            )
        )
    fig.update_layout(
        title = title,
        xaxis_title = "wavelengths [nm]" if axes_titles else "",
        yaxis_title = "intensity [A.u.]" if axes_titles else "")
    return fig

In [ ]:
# indicies of one spectrum from each class
class_representatives = [100, 1588, 2502, 3102, 3502, 4401, 5201, 6102, 6203, 7700, 9201, 9999]

In [ ]:
fig = plot_spectra(x_train_raw.loc[class_representatives].to_numpy(), wavelengths,
                    "Raw train spectra (Number of features: {})".format(x_train_raw.shape[1]))
fig.show()

Explore various interactive tools that plotly offers. E.g. disable/enable specific spectrum, zoom, download plot as png etc.

# 3) Preprocessing

It is usually not possible to use raw data directly taken from a measurement. Most of the machine learning methods need some preprocessing.

The basic preprocessing routine could be composed of: data normalization, outlier detecting/filtering, data labeling (separation to classes, ...) and more.
Here, we demonstrate simple 0-1 normalization and labeling of spectroscopic data for later classification.

## Normalization

Generally, normalization of the data is a crucial step before you can start building (training) ML models. The central idea of the normalization is to keep variables in a selected range, to be comparable together and also between individual data samples. Unnormalized data could lead to slow (or non-) convergence of the model (e.g. vanishing/exploding gradients in ANN, ...).
The basic advice is to keep your data with zero mean unit variance. However, this could be sometimes altered by the needs of an actual task (e.g spectroscopy: spectra are not zero centered).  

In spectroscopy, there are several commonly used normalization techniques:
 - 0-1 normalization (UVN) is used for classification tasks or more general qualitative analysis, where exact line intensity is not required

 - normalization to total emissivity could be beneficial for simple quantitative analysis (e.g. suppressing saturation effect in a calibration curve)

 - normalization to internal standard = spectral line (area of the peak)

In [ ]:
from sklearn.preprocessing import normalize

In [ ]:
norm = "max"
x_train_norm = pd.DataFrame(normalize(x_train_raw, norm=norm), index=x_train_raw.index, columns=wavelengths)
x_val_norm = pd.DataFrame(normalize(x_val_raw, norm=norm), index=x_val_raw.index, columns=wavelengths)

_Warning:_ Next cell calls `describe()` method. It can be time consuming.

In [ ]:
print("Raw dataset\n{}\n".format(x_train_raw.head()))
print("Normalized dataset\n{}".format(x_train_norm.head()))
print(x_train_norm.describe())

In [ ]:
fig = plot_spectra(x_train_norm.loc[class_representatives].to_numpy(), wavelengths,
                   "Normalized train spectra")
fig.show()

## One-hot encoder

In multiclass classification task, it is a good practice to convert class labels (1, 2, 3,...) into [one-hot](https://en.wikipedia.org/wiki/One-hot) labels (aka dummy). The reason is to preserve same differential between any pair of different classes. This is needed when calculating an error of a model.

---

_Note:_ 8 - 2 = 6 and 8 - 5 = 3 but is class 2 actually more different from class 8 than class 5? We cannot say without any additional information. With three classes encoded into one-hot encoding we obtain labels: [1, 0, 0,], [0, 1, 0] and [0, 0, 1], so their difference is always of size 1.

In [ ]:
y_train_onehot = pd.get_dummies(y_train_raw)
y_val_onehot = pd.get_dummies(y_val_raw)

In [ ]:
print("Numerical labels:\n{}\n".format(y_train_raw))
print("One-hot encoded labels:\n{}".format(y_train_onehot))

# 4) Dimensionality reduction
Often, in modern data science, the dimension (number of variables) of the data is very high. Combined with thousands (up to millions) of data samples it can be challenging to even process the data. Everyone can think of the increased computational cost of such data, but there are more obstacles. Our intuition and imagination are not working anymore in high-dimension and more, we cannot rely on common proved tools used in low dimension. Collectively, this is called "curse of the dimensionality" and a good example is not usability of euclidean distance measure in the higher dimension. \[[wikipedia](https://en.wikipedia.org/wiki/Curse_of_dimensionality)]

In modern spectroscopy, we collect large complex datasets where manual inspection (of each spectrum individually) is not possible. Thus, we need to automatically (or manually) select important spectral regions or somehow extract special features to carry useful information.

Most basic (naïve) dimension reduction is cropping out unimportant spectral regions. However, this technique is often time-consuming and requires advanced spectroscopic expertise (for the decision of correct spectral regions). The goal of ML is to automatize such processes, so we demonstrate several techniques to do so. Starting from a simple peak finder algorithm to more advanced Random Forest (RF) feature selection. As an alternative, feature extraction techniques are shown. In feature extraction, contrary to the selection, we create new variables to demonstrate original ones in a more efficient way. A good example is Principal Component Analysis (PCA), where we search for new variables composed as a linear combination of original variables in a way to maximize the explained variance of the data.


## Cropping
As was mentioned, cropping could be used for the selection of specific spectral regions (peaks). Here we use cropping to cut off the left and right side of spectra, where spectrometer sensitivity was low and information carried is negligible. You change the range by editing corresponding threshold variables.

In [ ]:
wl_idx_from = 2500
wl_idx_to = 33500
x_train_cropp = x_train_norm.iloc[:, wl_idx_from:wl_idx_to]
x_val_cropp = x_val_norm.iloc[:, wl_idx_from:wl_idx_to]
wavelengths_cropp = wavelengths[wl_idx_from:wl_idx_to]
print("Train dataset (cropped):\n{}\n".format(x_train_cropp))
print("Cropped wavelengths:\n{}".format(wavelengths_cropp))

## Feature selection

### Peak finder

To implement the automatized peak finder, we have used the [SciPy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html) library. For details read the documentation. Below, we search for peaks with `prominence` threshold 0.05 in the first spectrum (`spectrum_idx = 0`) of the dataset. You can try to adjust the prominence parameter value and see the results.

Later, try to explore the effect of other parameters of the  `find_peaks()` function.

In [ ]:
from scipy.signal import find_peaks

In [ ]:
spectrum_idx = 0
peaks, _ = find_peaks(x_train_cropp.iloc[spectrum_idx], prominence=0.05)

fig = plot_spectra([x_train_cropp.iloc[spectrum_idx]], wavelengths_cropp, labels=["spectrum"])
fig.add_trace(
    go.Scatter(
        x = wavelengths_cropp[peaks],
        y = x_train_cropp.iloc[spectrum_idx, peaks],
        name = "peaks",
        mode = "markers"
    )
)
fig.update_layout(
    title = "Number of detected peaks in spectrum #{}: {}".format(spectrum_idx, len(peaks)),
    autosize = True
)
fig.show()

Applying this algorithm to each spectrum in training data subset will allow us to obtain simple statistics of the dataset.

In [ ]:
peak_frequency = np.zeros(wavelengths_cropp.shape, dtype=int)
for _, spectrum in x_train_cropp.iterrows():
    spectrum_peaks, _ = find_peaks(spectrum, prominence=0.05)
    for peak_location in spectrum_peaks:
        peak_frequency[peak_location] += 1
peak_frequency = pd.Series(peak_frequency, dtype=int)

print("Statistics of 'votes' for peaks at particular position:\n{}".format(
    peak_frequency.describe(percentiles=[.25, .30, .50, .60, .70, .75])))

Now, we may filter out "unimportant" peaks. The selection criterion here is the total number of votes (over all spectra in the training dataset) per selected wavelength (variable). In other words, we select only wavelengths where we detect peaks more frequently.

In [ ]:
min_votes = 45  # Based on statistics from the cell above (circa 75th percentile)

peak_filter = [x >= min_votes for x in peak_frequency]
peak_filter = pd.Series(peak_filter, dtype=bool)
print("Number of wavelengths detected as peaks ('True') vs the rest ('False'):\n{}".format(
    peak_filter.value_counts()))

We created a filter (or mask) pd.Series containing `True` at wavelength's position which should be kept and `Flase` at positions which should be filtered out.

In [ ]:
fig = plot_spectra([np.mean(x_train_cropp)], wavelengths_cropp, labels=["mean spectrum"])
fig.add_trace(
    go.Scatter(
        x = wavelengths_cropp[peak_filter.values],
        y = np.mean(x_train_cropp)[peak_filter.values],
        name = "peaks",
        mode = "markers"
    )
)
fig.update_layout(
    title = "Number of detected peaks (throughout whole training dataset) with mean spectrum: {}"
                .format(peak_filter.value_counts()[True]),
    autosize = True
)
fig.show()

You can see that the the number of features dropped significantly. But there still is a lot of wavelengths marked as important (peaks) that probably shouldn't be marked and rather be dropped. You can try to change parameters of the peak-finder algorithm to get better results. Don't hesitate to contact us if you find better settings.

In [ ]:
x_train_peaks = x_train_cropp.loc[:, peak_filter.values]
x_val_peaks = x_val_cropp.loc[:, peak_filter.values]
wavelengths_peaks = wavelengths_cropp[peak_filter.values]

print("Train dataset (after peak finder): {}".format(x_train_peaks.shape))
print("Validation dataset (after peak finder): {}".format(x_val_peaks.shape))

In [ ]:
fig = plot_spectra(x_train_peaks.loc[class_representatives].to_numpy(), wavelengths_peaks,
            title="Dataset filtered by peak finder (# of features: {})".format(x_train_peaks.shape[1]))
fig.show()

### Extreme Random Forest Feature Selection

This method of feature selection is based on a classifier ([Extreme Random Forest / ExtraTree](https://scikit-learn.org/stable/modules/generated/sklearn.tree.ExtraTreeClassifier.html) in our case) that can natively describe importance of each feature for the classification result. We put _scikit-learn's_ [SelectFromModel](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html) on top of the classifier to filter out all features that are not important during the classification and keep only those with high impact (declared by the underlaying Extreme Random Forest (ERF)).

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

_Note_: Decision Trees and their derivatives such as ERF are not affected by not using one-hot encoded labels, so we can use raw labels here.

In [ ]:
clf = ExtraTreesClassifier(n_estimators = 200, verbose=1, n_jobs=-1)
selector = SelectFromModel(clf, prefit=False).fit(x_train_cropp, y_train_raw)

In [ ]:
erf_filter = pd.Series(selector.get_support(), dtype=bool)
print("Number of features (wavelengths) detected as important for classification ('True') vs the rest ('False'):\n{}".format(
    erf_filter.value_counts()))

In [ ]:
fig = plot_spectra([np.mean(x_train_cropp)], wavelengths_cropp, labels=["mean spectrum"])
fig.add_trace(
    go.Scatter(
        x = wavelengths_cropp[erf_filter.values],
        y = np.mean(x_train_cropp)[erf_filter.values],
        name = "important features",
        mode = "markers"
    )
)
fig.update_layout(
    title = "Number of important features: {}".format(erf_filter.value_counts()[True]),
    autosize = True
)
fig.show()

In [ ]:
x_train_erf = x_train_cropp.loc[:, erf_filter.values]
x_val_erf = x_val_cropp.loc[:, erf_filter.values]
wavelengths_erf = wavelengths_cropp[erf_filter.values]

print("Train dataset (after ERF-filter): {}".format(x_train_erf.shape))
print("Validation dataset (after ERF-filter): {}".format(x_val_erf.shape))

In [ ]:
fig = plot_spectra(x_train_erf.loc[class_representatives].to_numpy(), wavelengths_erf,
            title="Dataset filtered by ERF (# of features: {})".format(x_train_erf.shape[1]))
fig.show()

## Feature extraction

### Principal Component Aanalysis (PCA)
PCA is an unsupervised technique for dimensionality reduction and visualization of the data. It is often considered as the first choice for data exploration. The basic idea behind PCA is to find new variables (principal components - PCs) as a linear combination of the original variables with a constraint to maximize the variance explained by the PC (+ orthogonality of PCs). Later, only PCs with the highest contribution to the total variance are kept while dropping the rest. This will reduce the dimension of the data and open new possibilities for visualization.

PCA was proven as a very capable technique in spectroscopy (LIBS). To learn how to use PCA for data processing directly, see the [scikit-learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html).

In [ ]:
from sklearn.decomposition import PCA

At first we calculate the PCA (on just cropped dataset) and keep first 50 features.

In [ ]:
pca = PCA(n_components=50).fit(x_train_cropp)

Then we observe a [scree plot](https://en.wikipedia.org/wiki/Scree_plot) to choose proper number of principal components to keep.

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = [x+1 for x in range(len(pca.explained_variance_))],
        y = pca.explained_variance_ratio_ * 100,
        mode = "lines+markers",
    )
)
fig.update_layout(
    title = "Explained vairance by Principal Components (scree plot)",
    xaxis_title = "PC",
    yaxis_title = "explained variance [%]"
)
fig.show()

It is slightly subjective where to make a cut. You basically look for an "elbow". Let's use 10 PCs in our case, but 9, 11 or 12 could be used as well.

---

Loadings and scores are two important parts of PCA. Briefly, loadings describe how were PCs computed from original variables and scores are coordinates of samples in the new PCA-space.

[Here](https://stats.stackexchange.com/questions/143905/loadings-vs-eigenvectors-in-pca-when-to-use-one-or-another) you can read more about loadings and their connection to eigenvectors and eigenvalues.

In [ ]:
pca = PCA(n_components=10)
scores_pca_train = pca.fit_transform(x_train_cropp)
loadings = np.array([list(np.sqrt(pca.explained_variance_[i]) * pca.components_[i]) for i in range(10)])

It is possible to plot loadings same way as spectra.

In [ ]:
print("Scores:", scores_pca_train.shape)
print("Loadings:", loadings.shape)

fig = plot_spectra(loadings, wavelengths_cropp, "PCA loadigs",
            labels=["PC{}".format(x+1) for x in range(loadings.shape[0])])
fig.show()

From simplified point of view we can say that spectra with high PC1 value (in `scores`) are similar to the imaginary spectrum of PC1 (in `loadings`). Negative values in loadings denotes anticorrelation (absence of peaks at particular wavelength).

---

With low number of features (dimensions) we can finally plot samples into a human readable way and observe some hidden patterns in the dataset. Scores is the low-dimensional representation of our dataset, so let's plot them in a plane.

_Note:_ Try to change which PCs are plotted along x and y axis.

In [ ]:
x_axis = 1      # PC# along x axis
y_axis = 2      # PC# along y axis

fig = go.Figure()
for i_class in range(1, 13):
    fig.add_trace(
        go.Scatter(
            x = scores_pca_train[y_train_raw == i_class, x_axis-1],
            y = scores_pca_train[y_train_raw == i_class, y_axis-1],
            mode = "markers",
            name = "Class {}".format(i_class),
            marker = dict(
                size = 5,
                color = cl.scales['12']['qual']['Paired'][i_class-1]
            )
        )
    )
fig.update_layout(
    xaxis_title = "PC{}".format(x_axis),
    yaxis_title = "PC{}".format(y_axis),
    title = "PCA scores",
    width = 1000,
    height = 700,
    legend = dict(
        font = dict(
            size = 14
        ),
        itemsizing = 'constant'
    ),
)
fig.show()

Sometimes 2D is not enough and some pattern (e.g. two classes are [lineary separable](https://en.wikipedia.org/wiki/Linear_separability)) can be seen only by adding an extra dimension. Next scatter plot is same as the one above but in 3D. Try to rotate it, zoom or disable/enable a class.

In [ ]:
x_axis = 1      # PC# along x axis
y_axis = 2      # PC# along y axis
z_axis = 3      # PC# along z axis

fig = go.Figure()
for i_class in range(1, 13):
    fig.add_trace(
        go.Scatter3d(
            x = scores_pca_train[y_train_raw == i_class, x_axis-1],
            y = scores_pca_train[y_train_raw == i_class, y_axis-1],
            z = scores_pca_train[y_train_raw == i_class, z_axis-1],
            mode = "markers",
            name = "Class {}".format(i_class),
            marker = dict(
                size = 3,
                color = cl.scales['12']['qual']['Paired'][i_class-1]
            )
        )
    )
fig.update_layout(
    scene = dict(
        xaxis_title = "PC{}".format(x_axis),
        yaxis_title = "PC{}".format(y_axis),
        zaxis_title = "PC{}".format(z_axis)
    ),
    legend = dict(
        font = dict(
            size = 16
        ),
        itemsizing = 'constant'
    ),
    title = "PCA scores",
    width=1000,
    height=900
)
fig.show()

In [ ]:
pca_columns = ["PC{}".format(x) for x in range(1, 11)]
x_train_pca = pd.DataFrame(scores_pca_train,
                           index = x_train_cropp.index,
                           columns = pca_columns)
x_val_pca = pd.DataFrame(pca.transform(x_val_cropp),
                         index = x_val_cropp.index,
                         columns = pca_columns)

print("Training dataset transformed into PCA space {}:\n{}\n".format(x_train_pca.shape, x_train_pca))

PCA also allows you to transform your scores back to the original high-dimensional space using the calculated loadings.

In [ ]:
x_train_pca_restored = pd.DataFrame(pca.inverse_transform(x_train_pca), index=x_train_cropp.index,
                                    columns=wavelengths_cropp)

print("Training dataset transformed back to original space:\n{}".format(x_train_pca_restored))

The main point is that you are not able to obtain completely same dataset back because you have dropped some of the information and kept only 10 principal components. You can see how it affects the spectrum in the plot below.

---

_First 10 PCs describe only important information from the original dataset, not noise. Thus, the restored spectrum has less noise than the original one._

In [ ]:
plot_spectra([x_train_cropp.iloc[0], x_train_pca_restored.iloc[0]], wavelengths_cropp,
             title="Reconstructed spectrum form PCA (10 principal components) vs original spectrum",
             labels=["original", "reconstructed"])

### t-distributed Stochastic Neighbor Embedding (t-SNE)

t-SNE is an unsupervised iterative method designed to find a projection into low-dimensional space (typically 2D or 3D) based on similarities and disimilarities of samples in high-dimensional space. Similar samples (close points) in high-dimensional space should be close also in low-dimensional representation. The same stands also for far points respectively.

Here you can find more info about t-SNE:
- [wikipedia](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding)
- [scikit-learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html)
- [author's webpage](https://lvdmaaten.github.io/tsne/)

In [ ]:
from sklearn.manifold import TSNE

Not surprisingly, t-SNE has some advantages and disadvantages compared to PCA. Disadvantages are for example:
- Higher computation time.
- Only one single-directed projection. It's not possible to transform back to the original space and it's not possible to use already computed mapping to project new (unseen) samples to the low-dimensional space.
- There is significantly more parameters to optimize. t-SNE is a non-deterministic (probability based) method, so even with same parameters you can obtain different results.

On the other hand, t-SNE is designed for visualization, so the result is often much "nicer" than using PCA. Clusters in the low-dimensional projection can be clearer and more separated from each other.

_Note: Some implementations of t-SNE use PCA (with 50-100 PCs) as the first step (initialization) for speed up the computation._

_Note2: We use ERF-filtered dataset here. Computing t-SNE from  higher-dimensional dataset (`_cropp` or even `_norm` would take too long)._

In [ ]:
tsne = TSNE(n_components=2, init="pca", verbose=1, n_jobs=-1)
scores_tsne_train = tsne.fit_transform(x_train_erf)

In [ ]:
print("t-SNE 'scores': {}".format(scores_tsne_train.shape))

fig = go.Figure()
for i_class in range(1, 13):
    fig.add_trace(
        go.Scatter(
            x = scores_tsne_train[y_train_raw == i_class, 0],
            y = scores_tsne_train[y_train_raw == i_class, 1],
            mode = "markers",
            name = "Class {}".format(i_class),
            marker = dict(
                size = 5,
                color = cl.scales['12']['qual']['Paired'][i_class-1]
            )
        )
    )
fig.update_layout(
    title = "t-SNE space",
    xaxis_title = "dimension 1",
    yaxis_title = "dimension 2",
    width = 1000,
    height = 800,
    legend = dict(
        font = dict(
            size = 12
        ),
        itemsizing = 'constant'
    ),
)
fig.show()

It is possible to compute t-SNE for 3-dimensional space.

In [ ]:
tsne_3d = TSNE(n_components=3, init="pca", verbose=1, n_jobs=-1, perplexity=50)
scores_tsne_train_3d = tsne_3d.fit_transform(x_train_erf)

In [ ]:
fig = go.Figure()
for i_class in range(1, 13):
    fig.add_trace(
        go.Scatter3d(
            x = scores_tsne_train_3d[y_train_raw == i_class, 0],
            y = scores_tsne_train_3d[y_train_raw == i_class, 1],
            z = scores_tsne_train_3d[y_train_raw == i_class, 2],
            mode = "markers",
            name = "Class {}".format(i_class),
            marker = dict(
                size = 3,
                color = cl.scales['12']['qual']['Paired'][i_class-1]
            )
        )
    )
fig.update_layout(
    title = "t-SNE (train dataset) space",
    width = 1000,
    height = 800,
    legend = dict(
        font = dict(
            size = 12
        ),
        itemsizing = 'constant'
    ),

)
fig.show()

As mentioned above, it's not possible to simply use the same already computed mapping and add new points (e.g. validation set) to the plot. You have to compute t-SNE separately for validation set. Due to nondeterminism of the method it's possible that the result will be different than the one computed from training set.

In [ ]:
val_tsne = TSNE(n_components=2, init="pca", verbose=1, n_jobs=-1)
scores_tsne_val = tsne.fit_transform(x_val_erf)

You can compare the plot below with the first one (training dataset - 2D). They look similar in general, but you can't combine them into one using same coordinates.

In [ ]:
print("t-SNE validation 'scores': {}".format(scores_tsne_val.shape))

fig = go.Figure()
for i_class in range(NUM_CLASSES):
    fig.add_trace(
        go.Scatter(
            x = scores_tsne_val[y_val_raw == i_class, 0],
            y = scores_tsne_val[y_val_raw == i_class, 1],
            mode = "markers",
            name = "Class {}".format(i_class),
            marker = dict(
                size = 5,
                color = colormap[i_class]
            )
        )
    )
fig.update_layout(
    title = "t-SNE (validation dataset) space",
    xaxis_title = "dimension 1",
    yaxis_title = "dimension 2",
    width = 1000,
    height = 800,
    legend = dict(
        font = dict(
            size = 12
        ),
        itemsizing = 'constant'
    ),
)
fig.show()

# 5) Classification

## Support Vector Machines (SVM)

First classifier we use in this notebook is SVM. This method tries to find a hyperplane (a line in 2D space) that separates two classes. In advance it's a hyperplane such that margins between it and the closest data points are maximal [[wiki](https://en.wikipedia.org/wiki/Support-vector_machine)][[documentation](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)].

### Hyper-parameter grid search

SVM has several parameters that has to be optimized for every task (for details see documentation). A naive and computationally demanding (but safe) optimization method is a grid search combined with a cross validation.

[Cross Validation](https://scikit-learn.org/stable/modules/cross_validation.html) (CV) is a very usefull method and definitely a good practice when you evaluate your model. Briefly, you split your training dataset into $k$ disjoint subsets and train $k$ models separately. In every so-called _fold_ you use one of the subsets for validation (different one in every fold) and $k$-1 subsets for training. The result is then obtained from set of results (one for every fold) as mean of them. It's less affected by ranndomness during dataset selection.

The combination of CV with grid search is straightforward. At first you define the grid of parameters that should be used. Then you compute mean accuracy of every possible combination of parameters using CV. In the end you choose the best combination of parameters and train the model at the whole dataset.

Grid Search CV [tutorial](https://scikit-learn.org/stable/modules/grid_search.html) and [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

It's time consuming to go through the whole grid, so we already pruned it. You can try different parts of the grid or even whole.

In [ ]:
parameters = [
    # {'kernel': ['linear'], 'C': [1, 10, 1000, 10000]},
    {'kernel': ['rbf'], 'C': [10e5, 10e6, 10e10],
     'gamma': ['scale', 'auto', 0.01, 0.001]},
    #{'kernel': ['poly'], 'C': [1, 10, 1000, 10000],
    # 'gamma': ['scale', 'auto', 0.01, 0.001], 'degree': [3, 4, 5]}
]
svm = SVC()
gscv = GridSearchCV(svm, parameters, n_jobs=-1, verbose=1).fit(x_train_pca, y_train_raw)

In [ ]:
print("Best params: {}".format(gscv.best_params_))
print("Best score: {:.2f} %".format(gscv.best_score_ * 100))

You can save the grid search results for better analysis.

In [ ]:
cv_results = pd.DataFrame(gscv.cv_results_)
cv_results.to_csv("svm_gridsearch.csv")

### Final training and evaluation

In [ ]:
svm = SVC(kernel='rbf', C=1e7, gamma=0.001).fit(x_train_pca, y_train_raw)
y_train_pred = svm.predict(x_train_pca)
y_val_pred = svm.predict(x_val_pca)

There are basically two ways how to evaluate a model - [cross validation](https://machinelearningmastery.com/k-fold-cross-validation/) or _train-val-test_ split. Cross validation is especially handy when your dataset is not large enough and hard splitting would make one of the datasets (train, validation or test) too small. We use _train-val_ split in this tutorial. Test dataset would be later used for final evaluation and comparison with other contestants in the EMSLIBS contest, our validation dataset is used for evaluation of different models (types, architectures, preprocessing used, etc.).

In this case it is crucial not to "contaminate" the validation (or test) set by using it during any preprocessing or training step. You should put it aside as soon as you create it, use it only for evaluation of your model and you must not declare your model's performance on any dataset you used during training if you do not use CV.

_Note: Even selecting optimal number of epochs can be considered as contamination. From that purist point of view, what is done in chapter **Multilayer perceptron (MLP)** is a bad practice. We should use another 'test' dataset for declaring the performance, if we find optimal number of epochs using validation dataset. Typical datasets when you don't use CV are: 'training' - used for exploration and training; 'validation' - used for optimizing parameters of models (number of epochs, number of layers and neurons, kernels and their parameters, etc.) and their comparison (SVM vs MLP, different MLP topologies, etc.); 'test' - for evaluation of your final and optimized model (never seen by the model before), this should be your declared performance._

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import plotly.figure_factory as ff

This method plots a [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix). For more info see [this](https://www.geeksforgeeks.org/confusion-matrix-machine-learning/) blog or [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

In [ ]:
def plot_conf_matrix(y_true, y_pred, normalize, dataset_name):
    z = confusion_matrix(y_true, y_pred, normalize=normalize)
    fig = ff.create_annotated_heatmap (
        z,
        colorscale='Blues',
        x = [str(i) for i in range(1, 13)],
        y = [str(i) for i in range(1, 13)],
        showscale=True,
        annotation_text = np.around(z, decimals=2),
        hoverinfo='z')
    title = "Confusion matrix - '" + dataset_name + "'"
    if normalize == "true":
        title += " (normalized)"
    fig.update_layout(
        title = title,
        xaxis_title = {
            'text': "predicted class",
            'font': {'size': 10}
        },
        yaxis_title = {
            'text': "true class",
            'font': {'size': 10}
        },
        xaxis = {
            'linecolor': 'black',
            'mirror': True
        },
        yaxis = {
            'linecolor': 'black',
            'mirror': True
        },
        width = 500,
        height = 500
    )
    fig.show()

In [ ]:
plot_conf_matrix(y_train_raw, y_train_pred, normalize="true", dataset_name="train")

In [ ]:
plot_conf_matrix(y_train_raw, y_train_pred, normalize=None, dataset_name='train')

In [ ]:
print("Training dataset classification report:\n{}".format(classification_report(y_train_raw, y_train_pred)))

In [ ]:
plot_conf_matrix(y_val_raw, y_val_pred, normalize="true", dataset_name="val")

In [ ]:
plot_conf_matrix(y_val_raw, y_val_pred, normalize=None, dataset_name="val")

In [ ]:
print("Validation dataset classification report:\n{}".format(classification_report(y_val_raw, y_val_pred)))

Due to basic random _train-val_ split the results are pretty good. If you create a validation dataset more similar to the original test set (see the mentioned [paper](https://www.nature.com/articles/s41597-020-0396-8)), the model would probably tend to overfit.

---

Overfitted model has perfect accuracy on training dataset, but poor on validation dataset. There are many reasons why your model can overfit. One of them is that a model has problem with all classes that have low frequency in the training dataset. This is not surprising and it can be solved by [oversampling](https://en.wikipedia.org/wiki/Oversampling_and_undersampling_in_data_analysis).

## Multilayer perceptron (MLP)

Feedforward neural network or as called Multilayer perceptron ([MLP](https://en.wikipedia.org/wiki/Multilayer_perceptron)) is a type of Neural network used mainly for supervised tasks (classification or regression).

It is built from layers of neurons (simple computation unit) stacked one on each other. Every layer is fully connected with its neighbors (one lower and one upper layer). These connections are called weights and they are the parameters that are "learned" during training mode. For more info about neural networks see [Deep Learning Book](https://www.deeplearningbook.org/).

There are several ways how to easily create a neural network in Python. You can try [scikit-learn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.neural_network) used above or one of Python frameworks specialized on neural networks such as ([Theano](http://deeplearning.net/software/theano/index.html), [PyTorch](https://pytorch.org/) or [Keras](https://keras.io/)). We use Keras combined with [TensorFlow](https://www.tensorflow.org/) as a backend in this tutorial (can be combined also with Theano).

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, InputLayer
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

Here we create a simple MLP with 256-64-32-12 topology and train it on ERF dataset (filtered using Extreme Random Forest). Let's use a bigger number of epochs (epoch = round; all training samples are seen by the net once), e.g. 30 and see how accuracy evolved during the time.

In [ ]:
model = Sequential(name="MLP")
model.add(Dense(256, activation='relu', input_shape=(x_train_erf.shape[1],), name="hidden_1"))
model.add(Dropout(rate=0.2, name="dropout_1"))
model.add(Dense(64, activation='relu', name="hidden_2"))
model.add(Dropout(rate=0.2, name="dropout_2"))
model.add(Dense(32, activation='relu', name='hidden_3'))
model.add(Dropout(rate=0.2, name='dropout_3'))
model.add(Dense(12, activation='softmax', name="output"))
model.build()

model.summary()

model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

n_epochs = 50
history_obj = model.fit(x_train_erf, y_train_onehot,
                        batch_size=16,
                        epochs=n_epochs,
                        verbose=1,
                        validation_data=(x_val_erf, y_val_onehot))
history = pd.DataFrame(history_obj.history, index=[i+1 for i in range(n_epochs)])

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = history.index,
        y = history["accuracy"],
        mode = "lines+markers",
        name = "Train"
    )
)
fig.add_trace(
    go.Scatter(
        x = history.index,
        y = history["val_accuracy"],
        mode = "lines+markers",
        name = "Validation"
    )
)
fig.update_layout(
    title = "Model Accuracy",
    xaxis_title = "epochs",
    yaxis_title = "accuracy"
)
fig.show()

If you optimize your model too much to the training dataset, it is possible you fit it too much to it. It describes noise and some dataset-specific features instead of general patterns. This is called overfitting and you should always try to avoid this. In the image above, find the point where validation accuracy is still increasing and set the number of epochs to this number. Keep in mind that learning process is nondeterministic and several runs with totally same parameters can give different results.

_Note: Neural networks are very overfitting-prone_.

In [ ]:
y_train_pred = np.argmax(model.predict(x_train_erf), axis=-1) + 1
y_val_pred = np.argmax(model.predict(x_val_erf), axis=-1) + 1

In [ ]:
plot_conf_matrix(y_train_raw, y_train_pred, normalize="true", dataset_name="train")

In [ ]:
plot_conf_matrix(y_train_raw, y_train_pred, normalize=None, dataset_name='train')

In [ ]:
print("Training dataset classification report:\n{}".format(classification_report(y_train_raw, y_train_pred)))

In [ ]:
plot_conf_matrix(y_val_raw, y_val_pred, normalize='true', dataset_name='val')

In [ ]:
plot_conf_matrix(y_val_raw, y_val_pred, normalize=None, dataset_name='val')

In [ ]:
print("Validation dataset classification report:\n{}".format(classification_report(y_val_raw, y_val_pred)))